In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from CRUD_Module import *



###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name

username = "aacuser"
password = "weber"
shelter = animalShelter(username, password)

image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))
dfwr = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["Labrador Retriever Mix","Chesapeake Bay Retriever","Newfoundland"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))
dfmwr = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["German Shepherd","Alaskan Malamute","Old English Sheepdog","Siberian Huskey","Rottweiler"]},
             "sex_upon_outcome":"Intact Male",
             "age_upon_outcome_in_weeks":{"$gte":26},
             "age_upon_outcome_in_weeks":{"$lte":156}}))
dfdit = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":
             {"$in":["Doberman Pinscher","German Shepherd","Golden Retriever","Bloodhound","Rottweiler"]},
             "sex_upon_outcome":"Intact Female",
             "age_upon_outcome_in_weeks":{"$gte":20},
             "age_upon_outcome_in_weeks":{"$lte":300}}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Div(className='row',
        style={'display' : 'flex'},
             children=[
                html.I(html.H4('Rose Weber')),
                html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height' : '10%', 'width' : '10%'})
        ]),
#     html.Center(html.I(html.H4('Rose Weber'))),
#     html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'height' : '10%', 'width' : '10%'})),
    html.Hr(),
    dcc.Dropdown(
        id='dropdownFilter', 
        value='R', 
        options=[
            {'label': 'Water Rescue', 'value': 'WR'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'MWR'},
            {'label': 'Disaster or Individual Tracking', 'value': 'DIT'},
            {'label': 'Reset', 'value':'R'}
            ],
        clearable=False
    ),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id" : i, "deletable": False, "selectable": True} for i in df.columns],
        data = df.to_dict('records'),
        #FIXME: Set up the features for your interactive data table to make it user-friendly for your client
        style_cell={'textAlign': 'left'},
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(className='row',
        style={'display' : 'flex'},
             children=[
        html.Div(
            id='chart-id',
            className='col s12 m6',
            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
])

#############################################
# Interaction Between Components / Controller
#############################################

@app.callback(
    [Output('datatable-id','data'),
     Output('datatable-id','columns')],
    [Input('dropdownFilter', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
    if filter_type == 'R':
        new_df = df
    elif filter_type == 'WR':
        new_df = dfwr
    elif filter_type == 'MWR':
        new_df = dfmwr
    elif filter_type == 'DIT':
        new_df = dfdit
    else:
        new_df = df
        
    columns=[{"name": i, "id" : i, "deletable": False, "selectable": True} for i in df.columns]
    data = new_df.to_dict('records')
    return (data, columns)

#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')])
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])                               
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    if dff.iloc[0,9] == "":
        message = "No Name Found"
    else:
        message = dff.iloc[0,9]
        
    return [
        dl.Map(style={'width': '500px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(message)
                ])
            ])
        ])
    ]

@app.callback(
    Output('chart-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # convert data to pandas dataframe
    dff = pd.DataFrame.from_dict(viewData)
    # count by breed occurrence
    new_df = dff.breed.value_counts()
    new_df.name = "breed"
    new_df = new_df.to_frame()
    new_df.reset_index(inplace=True)

    # return updated graph data
    return [
        dcc.Graph(
            figure = px.pie(new_df, values="breed", names="index")
        )
    ]

app

Module Initialized
